In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Install Libraries

In [ ]:
# Installing Libraries
!sudo apt-get install python3.7

! pip install transformers -q
! pip install annoy -q

# Importing Libraries


from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd
import numpy as np
import string
import re
from tqdm.auto import tqdm
tqdm.pandas()

import _pickle as cPickle

# Checking to see if CUDA is available if so storing details in device


device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpython3.7-minimal libpython3.7-stdlib mailcap mime-support
  python3.7-minimal
Suggested packages:
  python3.7-venv binfmt-support
The following NEW packages will be installed:
  libpython3.7-minimal libpython3.7-stdlib mailcap mime-support python3.7
  python3.7-minimal
0 upgraded, 6 newly installed, 0 to remove and 18 not upgraded.
Need to get 4,698 kB of archives.
After this operation, 17.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 mailcap all 3.70+nmu1ubuntu1 [23.8 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 mime-support all 3.66 [3,696 B]
Get:3 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 libpython3.7-minimal amd64 3.7.17-1+jammy1 [608 kB]
Get:4 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 python3.7-minima

In [ ]:
! pip3 install syntok -q
! pip install -U sentence-transformers -q
! pip install langdetect -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
#Mean Pooling - Take attention mask into account for correct averaging


def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

In [ ]:
# # Function to generate embeddings

def get_embedings(model,sentences):
  model.to(device)   ##to run on gpu
  sent_len=len(sentences)
  limit=250
  if sent_len>limit:
    sentence_embeddings=[]
    for i in range(0,sent_len,limit):
      j=i+limit
      if j>sent_len:
        j=sent_len
      #Tokenize sentences   Here tokenize and genrtae sentence embeddings in 10000 chunk
      encoded_input = tokenizer(sentences[i:j], padding=True, truncation=True, return_tensors='pt')
      encoded_input.to(device)
      #Compute token embeddings
      with torch.no_grad():
          model_output = model(**encoded_input)
      #Perform pooling. In this case, mean pooling
      sentence_embeddings1 = mean_pooling(model_output, encoded_input['attention_mask'])
      sentence_embeddings = sentence_embeddings+sentence_embeddings1.tolist()
  else :
      encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
      encoded_input.to(device)
      #Compute token embeddings
      with torch.no_grad():
          model_output = model(**encoded_input)
      #Perform pooling. In this case, mean pooling
      sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
      sentence_embeddings = sentence_embeddings.tolist()
  print(len(sentence_embeddings))
  return sentence_embeddings

In [ ]:
# Defining the model here, we use a model that is fine tuned on News articles
# model_name = "/content/drive/Shareddrives/[FYP] Fake News Detection/Kirishnni Emotion Extraction/Resources/go_model"
model_name = "vinai/bertweet-base"

# Load AutoModel from huggingface model repository
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


In [ ]:
# df = pd.read_csv(r"/content/drive/Shareddrives/FYP - knk/Resources/NRC-Emotion-Intensity-Lexicon-v1.txt",  delimiter = "\t")
df = pd.read_csv(r"/content/drive/MyDrive/Krishnin/Resources/NRC Emotion Intensity.csv")

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9921 entries, 0 to 9920
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   word                     9921 non-null   object 
 1   emotion                  9921 non-null   object 
 2   emotion-intensity-score  9921 non-null   float64
 3   embedding                9921 non-null   object 
dtypes: float64(1), object(3)
memory usage: 310.2+ KB


In [ ]:
words = df['word'].values.tolist()
print(words)

['outraged', 'brutality', 'hatred', 'hateful', 'terrorize', 'infuriated', 'violently', 'furious', 'enraged', 'furiously', 'screwyou', 'murderer', 'fury', 'execution', 'angered', 'savagery', 'slaughtering', 'veryangry', 'assassinate', 'fuckoff', 'annihilation', 'rage', 'loathe', 'damnation', 'roadrage', 'fucktard', 'homicidal', 'furor', 'hostile', 'annihilate', 'murder', 'raging', 'sopissed', 'pissed', 'rape', 'explosive', 'obliterated', 'vengeful', 'ferocious', 'infuriates', 'killing', 'combative', 'gofuckyourself', 'vengeance', 'wrath', 'torment', 'vicious', 'threatening', 'massacre', 'bloodthirsty', 'abhorrent', 'pissoff', 'fighting', 'annihilated', 'attacking', 'angriest', 'bloodshed', 'smite', 'brawl', 'assault', 'assassination', 'strangle', 'explode', 'malicious', 'tirade', 'hostility', 'loathsome', 'attack', 'hell', 'murderous', 'malice', 'terrorism', 'beating', 'pissingmeoff', 'desecration', 'outrage', 'irate', 'tumultuous', 'destroying', 'violent', 'stab', 'infuriate', 'slaught

In [ ]:
emotion_embeddings = get_embedings(model, words)

9921


In [ ]:
df['embedding'] = emotion_embeddings

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9921 entries, 0 to 9920
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   word                     9921 non-null   object 
 1   emotion                  9921 non-null   object 
 2   emotion-intensity-score  9921 non-null   float64
 3   embedding                9921 non-null   object 
dtypes: float64(1), object(3)
memory usage: 310.2+ KB


In [ ]:
df.to_csv(r"/content/drive/MyDrive/Krishnin/Resources/NRC Emotion Intensity Emb.csv", index=False ,encoding='utf-8')

In [ ]:
dff = pd.read_csv(r"/content/drive/MyDrive/Krishnin/Resources/NRC Emotion Intensity Emb.csv")

In [ ]:
import ast

dff['embedding'] = [ast.literal_eval(i) for i in dff['embedding'].values.tolist()]
dff.head()

,word,emotion,emotion-intensity-score,embedding
0,outraged,anger,0.964,"[-0.13173015415668488, 0.1441633105278015, 0.1..."
1,brutality,anger,0.959,"[-0.16712629795074463, 0.1456366330385208, 0.1..."
2,hatred,anger,0.953,"[-0.14905360341072083, 0.11977861076593399, 0...."
3,hateful,anger,0.940,"[-0.16983617842197418, 0.15502475202083588, 0...."
4,terrorize,anger,0.939,"[0.04510786011815071, 0.093549445271492, 0.192..."


In [ ]:
word_embeddings = dff['embedding']

In [ ]:
! pip install annoy -q

In [ ]:
from annoy import AnnoyIndex

f = 768
t = AnnoyIndex(f, 'angular')  # Length of item vector that will be indexed
for i in range(len(word_embeddings)):
    t.add_item(i, word_embeddings[i])

t.build(500) # 500 trees

t.save(r"/content/drive/MyDrive/Krishnin/Resources/NRC Emotion Intensity Emb.ann")

True

In [ ]:
t.get_n_items()

9921